The cell below defines functions to calculate the consumption path for CRRA utility. You need to run this cell before any of the code below will work. 

In [1]:
using Optim
function u(c, ρ)
    if ρ==1
        return log(c)
    else
        return (c^(1-ρ)-1)/(1-ρ)
    end
end

function consumption(a0, y, m, β, δ, ρ)
    current_budget = a0 + y[1] -m[1]
    #println(length(y))
    if length(y) != length(m)
        return -Inf
    elseif length(y)==1
        return [current_budget]
    elseif current_budget <0
        return [-Inf]
    else
        next_y = y[2:end]
        next_m = m[2:end]
        next_c(c) = consumption(current_budget-c, next_y, next_m, β, δ, ρ)
        function utility(c)
            next_c_vec = next_c(c)
            if c<0 || minimum(next_c_vec) < 0
                return Inf
            end
            U = u(c, ρ)+ β * sum([δ^t * u(next_c_vec[t], ρ) for t in 1:length(next_c_vec)])
            return -U
        end
        if length(y)==5
            c_opt = optimize(utility, 0, current_budget, store_trace= true)
        else
            c_opt = optimize(utility, 0, current_budget)
        end
        c_star = Optim.minimizer(c_opt)
        c_vec =  next_c(c_star)
        pushfirst!(c_vec, c_star)
        return c_vec
    end
end

consumption (generic function with 1 method)

The cell below sets parameter values, and then calculates a consumption path based on those parameters. You can choose values for β, δ, and ρ (the CRRA coefficient). a0 is the assets that the agent starts with. y and m are respectively vectors of the income payments received at the start of each period, and the mortgage payments made at the start of each period. These vectors need to be the same length as each other (corresponding to the number of periods you want to model). Any length up to 5 runs nearly instantly, but 6 or more will take time to run. Run this cell every time you want to change the parameter values.

In [2]:
β = 0.6
δ = 0.95
ρ = 1.5
a0 = 0
y = [1000, 0, 1000, 0, 0]
m = [0, 0, 0, 400, 0]

c = consumption(a0, y, m, β, δ, ρ)

5-element Vector{Int64}:
   0
   0
   0
 400
   0

The cell below can be used to calculate whatever patterns in consumption you want to check by accessing elements of the vector c defined above. As an example, I've set it up to calculate the proportional slope in consumption between the first and second period, c_2 / c_1, and the slope between the third and fourth period, c_4/c_3.

In [11]:
println(c[2]/c[1])
println(c[4]/c[3])

0.9009839164813276
0.8533690291053004
